# Load libraries and data

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
df = pd.read_csv('The House Of Smooth Curry Review.csv')

In [4]:
df.tail()

,Review ID,Review
10,11,อาหารอร่อยแบบไทยโบราณภาคกลาง บรรยากาศดีมาก ตกแ...
11,12,อาหารรสชาติดี บรรยากาศดี พนักงานสุภาพ บริการดี...
12,13,บรรยากาศดีมากมีความเป็นไทยสูง เหมือนชาววัง อาห...
13,14,เพิ่งเคยกินเมี่ยงคำครั้งแรก ประทับใจเลยค่ะ รสช...
14,15,บริการดี อาหารอร่อย พนักงานมีเซอร์ไพร์สให้ด้วย...


# Tokenize Words

In [5]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_word = [' ','  ','\n','ร้าน', '(', ')','ค่ะ','ครับ']
screening_words = stopwords + removed_word

In [6]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine = 'newmm')
  for word in words:
      if word not in screening_words:
        merged = merged + ',' + word
  return merged[1:]

In [7]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [8]:
df.tail()

,Review ID,Review,Review_tokenized
10,11,อาหารอร่อยแบบไทยโบราณภาคกลาง บรรยากาศดีมาก ตกแ...,"อาหาร,อร่อย,ไทย,โบราณ,กลาง,บรรยากาศ,ดีมาก,ตกแต..."
11,12,อาหารรสชาติดี บรรยากาศดี พนักงานสุภาพ บริการดี...,"อาหาร,รสชาติ,ดี,บรรยากาศ,ดี,พนักงาน,สุภาพ,บริก..."
12,13,บรรยากาศดีมากมีความเป็นไทยสูง เหมือนชาววัง อาห...,"บรรยากาศ,ดีมาก,ไทย,เหมือน,ชาววัง,อาหาร,ตกแต่ง,..."
13,14,เพิ่งเคยกินเมี่ยงคำครั้งแรก ประทับใจเลยค่ะ รสช...,"กิน,เมี่ยงคำ,ครั้งแรก,ประทับใจ,รสชาติ,อร่อย,กล..."
14,15,บริการดี อาหารอร่อย พนักงานมีเซอร์ไพร์สให้ด้วย...,"บริการ,ดี,อาหาร,อร่อย,พนักงาน,เซอร์,ไพร,์ส,บรร..."


# Create Ditionary

In [9]:
documents = df['Review_tokenized'].to_list()
text = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(text)

In [10]:
print(dictionary.token2id.keys())

dict_keys(['1', 'drink', 'welcome', 'ขนม', 'คำแนะนำ', 'ซี่โครง', 'ดี', 'บริการ', 'พนักงาน', 'ฟรี', 'สุภาพ', 'หมู', 'อร่อย', 'อาหาร', 'เมนู', 'เลือก', 'เสิร์ฟ', 'โดยเฉพาะ', 'โต้', 'ให้ทาน', 'ได้ดี', 'งาน', 'ดีมาก', 'ทำอาหาร', 'นิ้ม', 'บรรยากาศ', 'พ', 'ม', 'รัก', 'วัตถุดิบ', 'สมราคา', 'เชฟ', 'แจ่มใส', 'แจ้', 'คอย', 'น้ำ', 'พื้นที่', 'รสชาติ', 'รองรับ', 'ร้องขอ', 'เติม', 'ใส่ใจ', 'ไม่ต้อง', 'ตกแต่ง', 'พนักงานบริการ', 'สดชื่น', 'สวยงาม', 'สะอาด', 'ไทย', 'ชัช', 'ดูแล', 'น่ารัก', 'น้อง', 'รน', 'ศิร', 'เท', 'กลั่นกรอง', 'กา', 'ก้าว', 'ครบรอบ', 'จาน', 'ต้น', 'ประทับใจ', 'มาจาก', 'รบ', 'รายละเอียด', 'ริก', 'รู้', 'ลืม', 'หัวใจ', 'เข้ามา', 'เริศ', 'แต่งงาน', 'โดน', 'ใจ', 'ใส', 'กลมกล่อม', 'ครอบครัว', 'ทาน', 'ผู้ใหญ่', 'หรู', 'เป็นแนว', 'เหมาะกับ', 'โมเดิร์น', 'กก', 'พูดจา', 'ย', 'ยย', 'แนะนำ', 'กะทิ', 'กุ้ง', 'คุ้มค่า', 'ตัว', 'สด', 'เลี่ยน', 'แกง', 'โดยรวม', 'ความรู้สึก', 'ดีเยี่ยม', 'ผู้จัดการ', 'รับประทาน', 'อบอุ่น', 'เหมาะสำหรับ', 'กลาง', 'วุ่นวาย', 'เกินงาม', 'เป็นกันเอง', 'โบราณ', 'กลับมา'

In [11]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in text]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

# Topic Modeling

In [12]:
num_topics=30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1 # do not evaluate model perplexity, takes too much time

# make a index to word dictionary
temp = dictionary[0]  # this is only to load the dictionary
id2word = dictionary.id2token

model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                      alpha='auto', eta='auto', \
                      iterations=iterations, num_topics=num_topics, \
                      passes=passes, eval_every=eval_every)



Visualize Result


In [13]:
vis_data = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds='mmds')

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [14]:
vis_data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.263283  0.040615       1        1  14.115594
15     0.226034  0.143831       2        1  13.714285
2     -0.212069  0.156161       3        1  10.917620
12    -0.257264 -0.060995       4        1   8.696298
6      0.061278 -0.254964       5        1   7.097554
8     -0.081611 -0.248114       6        1   7.097554
16    -0.182070 -0.159744       7        1   6.703308
5     -0.059519  0.250366       8        1   6.298437
20     0.088965  0.225279       9        1   6.298354
24     0.186244 -0.193091      10        1   5.898970
13     0.209414 -0.061627      11        1   5.898970
23    -0.138090  0.207110      12        1   5.499584
1     -0.005802 -0.002466      13        1   0.102043
26    -0.005811 -0.002492      14        1   0.097731
27    -0.005811 -0.002492      15        1   0.097731
22    -0.005811 -0.002492      16        1   0.097731
28    -0.005811 -0.002492      17        1   0.097731
21    -0.005811 -0.002492      18        1   0.097731
25    -0.005811 -0.002492      19        1   0.097731
0     -0.005811 -0.002492      20        1   0.097731
18    -0.005811 -0.002492      21        1   0.097731
17    -0.005811 -0.002492      22        1   0.097731
14    -0.005811 -0.002492      23        1   0.097731
11    -0.005811 -0.002492      24        1   0.097731
10    -0.005811 -0.002492      25        1   0.097731
9     -0.005811 -0.002492      26        1   0.097731
7     -0.005811 -0.002492      27        1   0.097731
4     -0.005811 -0.002492      28        1   0.097731
3     -0.005811 -0.002492      29        1   0.097731
29    -0.005811 -0.002492      30        1   0.097731, topic_info=        Term       Freq      Total Category  logprob  loglift
6         ดี  13.000000  13.000000  Default  30.0000  30.0000
22     ดีมาก   9.000000   9.000000  Default  29.0000  29.0000
12     อร่อย   9.000000   9.000000  Default  28.0000  28.0000
7     บริการ   8.000000   8.000000  Default  27.0000  27.0000
25  บรรยากาศ   8.000000   8.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25  บรรยากาศ   0.001823   8.381522  Topic30  -4.8978  -1.5024
26         พ   0.001823   1.141405  Topic30  -4.8978   0.4914
27         ม   0.001823   1.141405  Topic30  -4.8978   0.4914
28       รัก   0.001823   1.141405  Topic30  -4.8978   0.4914
29  วัตถุดิบ   0.001823   1.141405  Topic30  -4.8978   0.4914

[1611 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         4  0.852473         1
1         4  0.852473     drink
2         4  0.852473   welcome
84        5  0.876113        กก
76        7  0.529176  กลมกล่อม
...     ...       ...       ...
48        8  0.286803       ไทย
48       12  0.286803       ไทย
132       2  0.819442       ไพร
42        2  0.819442   ไม่ต้อง
133       2  0.819442        ์ส

[222 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 16, 3, 13, 7, 9, 17, 6, 21, 25, 14, 24, 2, 27, 28, 23, 29, 22, 26, 1, 19, 18, 15, 12, 11, 10, 8, 5, 4, 30])